In [7]:
import eikon as ek
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
ek.set_app_key('b499c15480534749a47879878c132348412d0c30')

get symbols from spx

In [9]:
data, _ = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [28]:
symbols = data['Symbol'].tolist()
symbols[:10]

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES']

the symbols come in the <span style="color:red">'ticker'</span> form, need to convert them to <span style="color:red">'RIC'</span>

In [83]:
ek.symbology.get_symbology(symbol='AAPL', from_symbol_type='ticker', to_symbol_type='RIC')

,error
AAPL,No best match available


In [82]:
ek.symbology.get_symbology(['US0378331005'], from_symbol_type='ISIN', to_symbol_type='RIC')

,RIC
US0378331005,AAPL.O


as seen above, for some reason, got to do it in two steps

<span style="color:red">'ticker'</span> -> <span style="color:red">'ISIN'</span> -> <span style="color:red">'RICS'</span>

In [189]:
def to_rics(symbols: list) -> list: 
    for i in symbols:
        try:
            i = ek.symbology.get_symbology(i, from_symbol_type='ticker', to_symbol_type='ISIN').values.tolist()
            i = ek.symbology.get_symbology(i, from_symbol_type='ISIN', to_symbol_type='RIC' ).values.tolist()
            rics.append(i)
        except Exception as e:
            print(e)
    return rics

In [ ]:
def to_cusip(symbols: list) -> list:
    

In [132]:
ek.symbology.get_symbology('ZBRA', from_symbol_type='ticker')

,CUSIP,ticker
ZBRA,989207105,ZBRA


In [153]:
i = ek.symbology.get_symbology(symbols, from_symbol_type='ticker', to_symbol_type='ISIN')

In [245]:
i['ISIN'].dropna()

MMM     US88579Y1010
ABT     US0028241000
ABBV    US00287Y1091
ABMD    US0036541003
ACN     IE00B4BNMY34
            ...     
XYL     US98419M1009
YUM     US9884981013
ZBH     US98956P1021
ZION    US9897011071
ZTS     US98978V1035
Name: ISIN, Length: 408, dtype: object

In [201]:
err = [j[0] for j in i.reset_index().values if not type(j[1]) is str]

In [204]:
np.array(err)  # np.array to save space

array(['AMD', 'ALB', 'ARE', 'ALL', 'AMCR', 'AAL', 'ADI', 'ANTM', 'AON',
       'APA', 'AMAT', 'APTV', 'ANET', 'ADP', 'BIIB', 'BA', 'BWA', 'AVGO',
       'BR', 'COG', 'CARR', 'CDW', 'CERN', 'CF', 'CI', 'CINF', 'CME',
       'COP', 'COST', 'CSX', 'DE', 'XRAY', 'FANG', 'DISH', 'DLTR', 'DOW',
       'EA', 'EVRG', 'ES', 'RE', 'EXC', 'FB', 'FAST', 'FOXA', 'FOX',
       'BEN', 'GRMN', 'IT', 'GL', 'HAS', 'PEAK', 'INFO', 'INTU', 'ISRG',
       'IPGP', 'SJM', 'JNPR', 'KEYS', 'KR', 'LB', 'MRO', 'MAR', 'MA',
       'MSFT', 'MNST', 'MCO', 'MS', 'MYL', 'NWL', 'NWSA', 'NWS', 'NI',
       'NBL', 'NTRS', 'ORCL', 'PCAR', 'PH', 'PEP', 'PM', 'PFG', 'PG',
       'REG', 'ROL', 'ROST', 'SPGI', 'SBAC', 'SLB', 'STX', 'SO', 'SNPS',
       'TXN', 'TSCO', 'VIAC', 'WBA', 'WELL', 'WST', 'ZBRA'], dtype='<U4')

___

ok this is lil complicated coz even though you can get pe for all the instruments and then get earninigs
from current price, yet different instruments need to be inputted in different ways  

for example, ABBV is ABBV.K at Eikon, can only be converted to OAPermID

In [260]:
ek.symbology.get_symbology('ABBV')

,OAPermID
ABBV,5037613143


In [261]:
ek.get_data('ABBV', ['EARNINGS', 'TR.PriceClose', 'TR.PE'])

(  Instrument  EARNINGS  Price Close  P/E (Daily Time Series Ratio)
 0       ABBV       NaN          NaN                            NaN,
 [{'code': 251658243,
   'col': 1,
   'message': "'The record could not be found' for the instrument 'ABBV'",
   'row': 0},
  {'code': 416,
   'col': 2,
   'message': "Unable to collect data for the field 'TR.PriceClose' and some specific identifier(s).",
   'row': 0},
  {'code': 416,
   'col': 3,
   'message': "Unable to collect data for the field 'TR.PE' and some specific identifier(s).",
   'row': 0}])

In [262]:
ek.get_data('5037613143', ['EARNINGS', 'TR.PriceClose', 'TR.PE'])

(   Instrument  EARNINGS  Price Close  P/E (Daily Time Series Ratio)
 0  5037613143       NaN        97.01                      17.165719,
 [{'code': 251658243,
   'col': 1,
   'message': "'The record could not be found' for the instrument '5037613143'",
   'row': 0}])

In [263]:
ek.get_data('ABBV.K', ['EARNINGS', 'TR.PriceClose', 'TR.PE'])

(  Instrument  EARNINGS  Price Close  P/E (Daily Time Series Ratio)
 0     ABBV.K    5.6514        97.01                      17.165719,
 None)

In [264]:
# price / earnings = 17.16
97/5.65

17.168141592920353

weird isnt it lol